## **`Introduction:`**
In today's digital age, online shopping has become the go-to option for many consumers. However, the overwhelming number of choices and the lack of personalized assistance can make the shopping experience daunting. To address this, we have developed ShopAssist AI, a chatbot that combines the power of large language models and rule-based functions to ensure accurate and reliable information delivery.

In ShopAssist AI, your chatbot to demonstrate the capability to:

1. Seamlessly interact with users to discern their intent and requirements.
2. Extract relevant information from the database based on user intent.
3. Present tailored recommendations to users.

**`Problem Statement:`**
Given a dataset containing information about laptops (product names, specifications, descriptions, etc.), build a chatbot that parses the dataset and provides accurate laptop recommendations based on user requirements.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
laptop_df = pd.read_csv('/content/drive/My Drive/laptop_data.csv')
laptop_df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [2]:
# Install OpenAI library
!pip install -U -q openai tenacity

**Approach:**
1. **Conversation and Information Gathering:**  The chatbot will utilize language
models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction:** Once the essential information is collected, rule-based functions come into play, extracting top 3 laptops that best matches the user's needs.
3. **Personalized Recommendation:** Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.

# **System Design**
We have a dataset laptop.csv where each row describes the features of a single laptop and also has a small description at the end. The chatbot that we build will leverage LLMs to parse this Description column and provide recommendations

**Workings of the Chatbot**

**Note1** - Don't run this when you go through this section for the first time. Run this only after running through the rest of the code. When going through this for the first time, please only go through the output, to understand the workings of the chatbot

**Note2** - The conversation shown here might be different from what you see in the video. However, the overall structure of the conversation to understand the system design remains the same.



dialogue_mgmt_system()

`**Import the libraries**`
Let's start by importing the libraries that we'll require for this project. Following are the ones:

openai
pandas
os, json, ast
Make sure the api key is stored in the text file OPENAI_API_Key.txt.

In [2]:
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [2]:
!pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.70.0
    Uninstalling openai-1.70.0:
      Successfully uninstalled openai-1.70.0


In [3]:
from google.colab import userdata
openai.api_key = userdata.get('OpenAI_Key')

In [4]:
## Function Description for the Function Calling API

function_descriptions = [
            {
                "name": "compare_laptops_with_user",
                "description": "Get the top 3 laptops from the catalogue, that best matches what the user is asking based on 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' & 'Budget",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "gpu intensity": {
                            "type": "string",
                            "description": "The requirement of the user in GPU capacity classfied as low, medium or high" ,
                        },
                        "display quality": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Display Quality & capacity classfied as low, medium or high" ,
                        },
                        "portability": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's portability classfied as low, medium or high" ,
                        },
                        "multitasking": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Multitasking classfied as low, medium or high" ,
                        },
                        "processing speed": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Processing speed classfied as low, medium or high" ,
                        },
                        "budget": {
                            "type": "integer",
                            "description": "The maximum budget of the user" ,
                        },

                    },
                    "required": ["GPU intensity","Display quality","Portability","Multitasking","Processing speed","Budget"],
                },
            }
        ]

In [5]:
def initialize_conversation():
    delimiter = "####"

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You are trying to understand the user's requirement for these laptop's features: ('gpu intensity','display quality','portability','multitasking','processing speed','budget')
    You need to ask relevant questions and understand the user need for each feature by analysing the user's responses.
    After understanding their requirements, you'll use a function call to suggest the top 3 laptops with their respective user match score.
    Recommend these laptops and answer any user's query about them.

    {delimiter} Here are certain guidelines that you need to follow:
    Don't ask questions about more than 2 features at a time.
    If the user's budget is less than says less than 25000 INR, please mention that there are no laptops in that range.
    Recommend the top3 laptops in the following format:
    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    3. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    {delimiter}

    {delimiter}To find the top3 laptops, you need to have the following chain of thoughts:
    Thought 1: Ask one question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    Answer "Yes" or "No" to indicate if you understand the requirements. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n

    Thought 2: Now, you are trying to understand the requirements for other features which you couldn't in the previous step.
    Ask questions to strengthen your understanding of the user's profile.
    Don't ask questions about more than 2 features at a time.
    Answer "Yes" or "No" to indicate if you understood all the needs of the features and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the features whose needs you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the feature you want to understand the need for.{delimiter}
    {delimiter}

    {delimiter} Here is a sample conversation between the user and assistant:
    Assistant: "Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?"
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    {delimiter}


    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

In [6]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You are trying to understand the user\'s requirement for these laptop\'s features: (\'gpu intensity\',\'display quality\',\'portability\',\'multitasking\',\'processing speed\',\'budget\')\n    You need to ask relevant questions and understand the user need for each feature by analysing the user\'s responses.\n    After understanding their requirements, you\'ll use a function call to suggest the top 3 laptops with their respective user match score.\n    Recommend these laptops and answer any user\'s query about them.\n\n    #### Here are certain guidelines that you need to follow:\n    Don\'t ask questions about more than 2 features at a time.\n    If the user\'s budget is less than says less than 25000 INR, please mention that there are no laptops in that range.\n    Recommend the top3 laptops in the following format:\n    Start with a brief summar

In [66]:
def get_chat_model_completions(messages):
    response = openai.ChatCompletion.create(
        model = "gpt-4-0613",
        messages = messages,
        temperature = 0,
        max_tokens = 500,
        functions = function_descriptions,
        function_call = "auto"
        )

    return response.choices[0].message["content"]

In [67]:
debug_introduction = get_chat_model_completions(debug_conversation)
print(debug_introduction)

Sure, I'd be happy to help you find a laptop for gaming. To better understand your needs, could you please tell me more about the types of games you play? Are they high-end games that require a powerful GPU? Also, what is your budget for this laptop?


In [11]:
debug_user_input = "Hi,Suggest me laptop for gaming."

In [12]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Sure, I'd be happy to help you find a laptop for gaming. To better understand your needs, could you please tell me more about the types of games you play? Are they high-end games that require a powerful GPU? Also, what is your budget for this laptop?


Typically, whenever the chatbot is interacting with the user, all the conversations should be moderated to identify any inappropriate content. Let's look at the function that can help with it.

`moderation_check():` This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, you can add a break statement to end the conversation.

In [13]:
def moderation_check(user_input):
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]
    print("Output from Moderation check: " + str(moderation_output))
    if moderation_output["flagged"] == True:
        return "Flagged"
    else:
        return "Not Flagged"

In [14]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Output from Moderation check: {
  "flagged": false,
  "categories": {
    "sexual": false,
    "hate": false,
    "harassment": false,
    "self-harm": false,
    "sexual/minors": false,
    "hate/threatening": false,
    "violence/graphic": false,
    "self-harm/intent": false,
    "self-harm/instructions": false,
    "harassment/threatening": false,
    "violence": false
  },
  "category_scores": {
    "sexual": 5.786332167190267e-06,
    "hate": 3.8982982175639336e-08,
    "harassment": 1.6806608300612425e-06,
    "self-harm": 0.00010218042734777555,
    "sexual/minors": 4.6085051508271135e-06,
    "hate/threatening": 5.9195976120918203e-08,
    "violence/graphic": 1.156620419351384e-05,
    "self-harm/intent": 0.001911655650474131,
    "self-harm/instructions": 0.0006166138919070363,
    "harassment/threatening": 6.963372243262711e-07,
    "violence": 3.3393193007213995e-05
  }
}
Not Flagged


In [15]:
#Let's now check moderation on the assistant's response debug_response_assistant.
moderation_check(debug_response_assistant)

Output from Moderation check: {
  "flagged": false,
  "categories": {
    "sexual": false,
    "hate": false,
    "harassment": false,
    "self-harm": false,
    "sexual/minors": false,
    "hate/threatening": false,
    "violence/graphic": false,
    "self-harm/intent": false,
    "self-harm/instructions": false,
    "harassment/threatening": false,
    "violence": false
  },
  "category_scores": {
    "sexual": 3.887183083861601e-06,
    "hate": 5.8073290176707815e-08,
    "harassment": 6.738476258760784e-06,
    "self-harm": 4.896523932984564e-07,
    "sexual/minors": 6.310802405096183e-07,
    "hate/threatening": 5.024750038273851e-08,
    "violence/graphic": 1.6643374465274974e-06,
    "self-harm/intent": 7.652444082850707e-07,
    "self-harm/instructions": 9.877783213596558e-07,
    "harassment/threatening": 3.3833734960353468e-06,
    "violence": 9.613459405954927e-05
  }
}


'Not Flagged'

In [20]:
print(moderation_check("I will kill Xyz "))


Output from Moderation check: {
  "flagged": true,
  "categories": {
    "sexual": false,
    "hate": false,
    "harassment": false,
    "self-harm": false,
    "sexual/minors": false,
    "hate/threatening": false,
    "violence/graphic": false,
    "self-harm/intent": false,
    "self-harm/instructions": false,
    "harassment/threatening": false,
    "violence": true
  },
  "category_scores": {
    "sexual": 6.374006716214353e-06,
    "hate": 0.0002631944080349058,
    "harassment": 0.07273797690868378,
    "self-harm": 6.42826398689067e-06,
    "sexual/minors": 4.8416470832535197e-08,
    "hate/threatening": 5.8295991038903594e-05,
    "violence/graphic": 1.4229048019842594e-06,
    "self-harm/intent": 2.2265062398219015e-06,
    "self-harm/instructions": 1.7076061453735747e-07,
    "harassment/threatening": 0.06432300060987473,
    "violence": 0.9835945963859558
  }
}
Flagged


In [18]:
print(moderation_check("what is 5+5 "))

Output from Moderation check: {
  "flagged": false,
  "categories": {
    "sexual": false,
    "hate": false,
    "harassment": false,
    "self-harm": false,
    "sexual/minors": false,
    "hate/threatening": false,
    "violence/graphic": false,
    "self-harm/intent": false,
    "self-harm/instructions": false,
    "harassment/threatening": false,
    "violence": false
  },
  "category_scores": {
    "sexual": 0.0005717585445381701,
    "hate": 1.3407323422143236e-05,
    "harassment": 8.348443770955782e-06,
    "self-harm": 6.573742211912759e-06,
    "sexual/minors": 0.0028667019214481115,
    "hate/threatening": 9.281470170208195e-07,
    "violence/graphic": 5.1050847105216235e-06,
    "self-harm/intent": 1.2067649549862836e-05,
    "self-harm/instructions": 7.294009378711053e-07,
    "harassment/threatening": 3.51689891431306e-07,
    "violence": 7.716290383541491e-06
  }
}
Not Flagged


In [21]:
def product_map_layer(laptop_description):

  delimiter = "#####"
  lap_spec = {
      "GPU intensity":"(Type of the Graphics Processor)",
      "Display quality":"(Display Type, Screen Resolution, Display Size)",
      "Portability":"(Laptop Weight)",
      "Multitasking":"(RAM Size)",
      "Processing speed":"(CPU Type, Core, Clock Speed)"
  }

  values = {'low','medium','high'}

  prompt=f"""
  You are a Laptop Specifications Classifier whose job is to extract the key features of laptops as per their requirements.
  To analyze each laptop, perform the following steps:
  Step 1: Extract the laptop's primary features from the description {laptop_description}
  Step 2: Store the extracted features in {lap_spec} \
  Step 3: Classify each of the items in {lap_spec}into {values} based on the following rules: \
  {delimiter}
  GPU Intensity: low: < for Integrated graphics or entry-level dedicated graphics like Intel UHD > , \n
  medium: < if Mid-range dedicated graphics like M1, AMD Radeon, Intel Iris > , \n
  high: < High-end dedicated graphics like Nvidia > , \n

  Display Quality: low: < if resolution below Full HD (e.g., 1366x768). > , \n
  medium: < if Full HD resolution (1920x1080) or higher. > , \n
  high: < if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. > \n

  Portability: low: < if laptop weight is less than 1.51 kg > , \n
  medium: < if laptop weight is between 1.51 kg and 2.51 kg> , \n
  high: <if laptop weight is greater than 2.51 kg> \n

  Multitasking: low: < If RAM size is 8GB, 12GB > , \n
  medium: < if RAM size is 16GB > , \n
  high: < if RAM size is 32GB, 64GB> \n

  Processing Speed: low: < if entry-level processors like Intel Core i3, AMD Ryzen 3> , \n
  medium: < if Mid-range processors like Intel Core i5, AMD Ryzen 5 > , \n
  high: < if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher > \n
  {delimiter}

  {delimiter}
  Here are some input output pairs for few-shot learning:
  input1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
  output1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

  input2: "The Lenovo ThinkPad X1 Carbon is a sleek and lightweight laptop designed for professionals on the go. It is equipped with an Intel Core i7 processor running at 2.6 GHz, providing strong processing capabilities for multitasking and productivity. With 16GB of RAM and an SSD, it offers fast and efficient performance along with ample storage capacity. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It comes with Intel UHD integrated graphics for decent graphical performance. Weighing just 1.13 kg, it is extremely lightweight and highly portable. The laptop features an IR camera for face unlock, providing convenient and secure login options. With a three-year warranty and an impressive battery life of up to 12 hours, the Lenovo ThinkPad X1 Carbon ensures reliability and long-lasting productivity. Priced at 130,000, it offers top-notch performance and portability for professionals."
  output2: {{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'high', 'Multitasking':'high', 'Processing speed':'high'}}

  input3: "The Apple MacBook Pro is a high-end laptop that combines top-tier performance with a stunning display. It is equipped with an Intel Core i9 processor running at 2.9 GHz, providing exceptional processing power for demanding tasks and content creation. With 32GB of RAM and an SSD, it offers seamless multitasking and fast storage access for large projects. The laptop features a 16" Retina display with a resolution of 3072x1920, delivering breathtaking visuals and precise color reproduction. It comes with an AMD Radeon graphics card, ensuring smooth graphics performance for professional applications. Weighing 2.02 kg, it is relatively lightweight for its size. The laptop features a True Tone display, adjusting the color temperature to match the ambient lighting for a more natural viewing experience. With a three-year warranty and a battery life of up to 10 hours, the Apple MacBook Pro offers reliability and endurance for professionals. Priced at 280,000, it caters to users who require uncompromising performance and a superior display for their demanding workloads."
  output3: {{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'medium','Multitasking': 'high', 'Processing speed': 'high'}}
  {delimiter}

  Follow the above instructions step-by-step and output the dictionary {lap_spec} for the following laptop {laptop_description}.
  """

#see that we are using the Completion endpoint and not the Chatcompletion endpoint

  response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens = 2000,
    # temperature=0.3,
    # top_p=0.4
    )
  return response["choices"][0]["text"]

In [22]:
import pandas as pd
import ast
import re

laptop_df = pd.read_csv('/content/drive/My Drive/laptop_data.csv')
## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))




In [23]:
laptop_df.head(5)


,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,laptop_feature
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"\n\n{'GPU intensity': 'medium',\n 'Display qua..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"\n{'GPU intensity': 'high', 'Display quality':..."
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"\n{'GPU intensity': 'medium', 'Display quality..."
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"\n{'GPU intensity': 'low', 'Display quality': ..."
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"\n{'GPU intensity': 'high', 'Display quality':..."


In [50]:
laptop_df['laptop_feature']

,laptop_feature
0,"\n\n{'GPU intensity': 'medium',\n 'Display qua..."
1,"\n{'GPU intensity': 'high', 'Display quality':..."
2,"\n{'GPU intensity': 'medium', 'Display quality..."
3,"\n{'GPU intensity': 'low', 'Display quality': ..."
4,"\n{'GPU intensity': 'high', 'Display quality':..."
5,"\n{'GPU intensity': 'high', 'Display quality':..."
6,"\n\n{'GPU intensity': 'medium', 'Display quali..."
7,"\n{'GPU intensity': 'high', 'Display quality':..."
8,"\n{'GPU intensity': 'low', 'Display quality':'..."
9,"\n {'GPU intensity': 'high', 'Display quality..."


In [52]:
laptop_df.to_csv("updated_laptop.csv",index=False,header = True)

In [48]:
import ast
import re





def extract_dictionary_from_string(string):
  regex_pattern = r"\{[^{}]+}"

  dictionary_matches = re.findall(regex_pattern, string)

  ## Extract the first dictionary:
  if dictionary_matches:
    dictionary_string = dictionary_matches[0]
    dictionary_string = dictionary_string.lower()
    dictionary = ast.literal_eval(dictionary_string)
    return dictionary # Return the dictionary if found
  else:
    return {} # Return an empty dictionary if no match is found

In [49]:
string = " \n \t           Output: {{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '75000'}}ad"
extracted_dict = extract_dictionary_from_string(string)
print(extracted_dict)

{'gpu intensity': 'high', 'display quality': 'high', 'portability': 'low', 'multitasking': 'high', 'processing speed': 'medium', 'budget': '75000'}


In [53]:


def compare_laptops_with_user(user_req_string):
    laptop_df= pd.read_csv('updated_laptop.csv')
    user_requirements = extract_dictionary_from_string(user_req_string)
    budget = int(user_requirements.get('budget', '0').replace(',', '').split()[0])
   # Extracting user requirements from the input string (assuming it's a dictionary)
   # # Creating a copy of the DataFrame and filtering laptops based on the budget
   # Since the function parameter already seems to be a string, we'll use it directly instead of extracting from a dictionary
   # Extracting the budget value from user_requirements and converting it to an integer


    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
    #These lines create a copy of the laptop_df DataFrame and assign it to filtered_laptops.
    #They then modify the 'Price' column in filtered_laptops by removing commas and converting the values to integers.
    #Finally, they filter filtered_laptops to include only rows where the 'Price' is less than or equal to the budget.

    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = extract_dictionary_from_string(user_product_match_str)
        score = 0

        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
                ### If the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1

        filtered_laptops.loc[index, 'Score'] = score

    # Sort the laptops by score in descending order and return the top 5 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

    return top_laptops.to_json(orient='records')



In [28]:
def recommendation_validation(laptop_recommendation):
  data = json.loads(laptop_recommendation)

  data1 = []
  for i in range(len(data)):
    if data[i]['Score'] > 2:
      data1.append(data[i])
  return json.dumps(data1)

In [74]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    print(introduction + '\n')
    top_3_laptops = None
    user_input = ''

    while(user_input != "exit"):
        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_3_laptops is None:
            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_model_completions(conversation)

            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            moderation = moderation_check(confirmation)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            if "No" in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
            else:
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
                response = dictionary_present(response_assistant)

                moderation = moderation_check(response)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                print('\n' + response + '\n')
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptops_with_user(response)

                validated_reco = recommendation_validation(top_3_laptops)

                if len(validated_reco) == 0:
                    print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
                    break

                conversation_reco = initialize_conv_reco(validated_reco)
                recommendation = get_chat_model_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "user", "content": "This is my user profile" + response})

                conversation_reco.append({"role": "assistant", "content": recommendation})

                print(recommendation + '\n')

        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_model_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

As mentioned earlier, you need to understand the user's profile, which essentially means that all the features: GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget are captured or not. Let's look at the function that helps us verify that.

intent_confirmation_layer(): This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
  GPU intensity,Display quality,
  Portability,
  Multitasking,
  Processing speed,
  Budget

In [30]:
def intent_confirmation_layer(response_assistant):
  delimiter = "####"
  prompt = f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: 'GPU intensity','Display quality','Portability','Multitasking',' Processing speed','Budget'
    Next you need to evaluate if the keys have the the values filled correctly.
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance as stated by user. The value for the key 'budget' needs to contain a number with currency.
    Output a string 'Yes' if the input contains the dictionary with the values correctly filled for all keys.
    Otherwise out the string 'No'.

    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """


  confirmation = openai.Completion.create(
                                    model="gpt-3.5-turbo-instruct",
                                    prompt = prompt,
                                    temperature=0)


  return confirmation["choices"][0]["text"]

In [31]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)


No


In [76]:
#testing the code done till here
dialogue_mgmt_system()

Assistant: "Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?"

gaming, High processing speed, good display, budget 80000
Output from Moderation check: {
  "flagged": false,
  "categories": {
    "sexual": false,
    "hate": false,
    "harassment": false,
    "self-harm": false,
    "sexual/minors": false,
    "hate/threatening": false,
    "violence/graphic": false,
    "self-harm/intent": false,
    "self-harm/instructions": false,
    "harassment/threatening": false,
    "violence": false
  },
  "category_scores": {
    "sexual": 8.449283632216975e-05,
    "hate": 8.850928679748904e-06,
    "harassment": 1.8799033568939194e-05,
    "self-harm": 2.0872310415143147e-05,
    "sexual/minors": 5.931367923039943e-06,
    "hate/threatening": 2.308697185071651e-05,
    "violence/graphic": 0.00011352120782248676,
    "self-harm/intent": 0.00021702998492401093,
    "self-harm/instructions": 0.0011917072115465999,
    "harassm

KeyboardInterrupt: Interrupted by user

In [33]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue: {products}.\
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

In [34]:
#Let's add the above assistant response to the debug_conversation.
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})

In [35]:
debug_response_assistant_n = f"""

Based on the information you have provided, I have updated the values in the dictionary as follows:

{{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low', 'Budget': '90000'}}

Please note that the values for 'Display quality' and 'Portability' are set to 'low' since they are not a priority for your coding tasks.

Is there anything else I can assist you with?
"""
#Note that you are using double curly braces

In [36]:
intent_confirmation_layer(debug_response_assistant_n)

'\nYes'

In [47]:
#Let's now look at the working of dictionary_present().
#dictionary_present(): This function checks if the final understanding of user's profile is returned by the chatbot is a Python dictionary or not.
#This is important as it'll be used later on for finding the right laptops using dictionary matching.


def dictionary_present(response):
    delimiter = "####"
    user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'}
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input: {{'GPU intensity':     'low', 'Display quality':     'high', 'Portability':    'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90,000'}}
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90000'}}

            input: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'low','Processing speed': 'high','Budget': '200000 INR'
            output: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'low','Budget': '200000'}}
            {delimiter}

            Here is the input {response}

            """
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens = 2000
        # temperature=0.3,
        # top_p=0.4
    )
    return response["choices"][0]["text"]




In [46]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)



{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '80000'}


In [39]:
debug_response_assistant_n = f"""Thank you for providing your budget. Based on your budget of 50,000 INR, I will consider this while recommending suitable laptop options for you.
Here is the final recommendation for your laptop:

- GPU intensity: high
- Display quality: high
- Portability: low
- Multitasking: high
- Processing speed: medium
- Budget: 80,000 INR

Please note that these specifications are based on your requirements for surfing and a decent display within your budget. Let me know if there's anything else I can assist you with!"""

In [45]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)


{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '80000'}


In [41]:
string = " \n \t           Output: {{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}ad"
extracted_dict = extract_dictionary_from_string(string)
print(extracted_dict)

{'gpu intensity': 'high', 'display quality': 'high', 'portability': 'low', 'multitasking': 'high', 'processing speed': 'medium', 'budget': '50000'}


In [72]:
top_3_laptops = compare_laptops_with_user(response_dict_n)
print(top_3_laptops)

[{"Brand":"MSI","Model Name":"GL65","Core":"i7","CPU Manufacturer":"Intel","Clock Speed":"2.6 GHz","RAM Size":"16GB","Storage Type":"HDD+SSD","Display Type":"IPS","Display Size":"15.6\"","Graphics Processor":"NVIDIA GTX","Screen Resolution":"1920x1080","OS":"Windows 10","Laptop Weight":"2.3 kg","Special Features":"RGB Keyboard","Warranty":"2 years","Average Battery Life":"4 hours","Price":55000,"Description":"The MSI GL65 is a high-performance laptop designed for gaming enthusiasts. Powered by an Intel Core i7 processor running at 2.6 GHz, it delivers exceptional processing power for smooth gaming and demanding tasks. With 16GB of RAM and a combination of HDD and SSD storage, it offers ample memory and fast data access. The laptop features a 15.6\" IPS display with a resolution of 1920x1080, ensuring vivid colors and wide viewing angles for an immersive gaming experience. Equipped with an NVIDIA GTX graphics card, it provides excellent visual performance and smooth gameplay. Weighing j

# **Product Recommendation Layer**
Finally, we come to the product recommendation layer. It takes the output from the compare_laptops_with_user function in the previous layer and provides the recommendations to the user. It has the following steps.

Initialize the conversation for recommendation.
Generate the recommendations and display in a presentable format.
Ask questions basis the recommendations.

In [56]:
debug_conversation_reco = initialize_conv_reco(top_3_laptops)

In [57]:
debug_recommendation = get_chat_model_completions(debug_conversation_reco)
print(debug_recommendation + '\n')

1. Acer Predator: Intel Core i7 processor clocked at 2.8 GHz, 16GB RAM, SSD storage, 17.3" IPS display with a resolution of 1920x1080, NVIDIA GTX graphics card, Windows 10 OS, weighs 3.2 kg, special features include dual cooling fans, 1-year warranty, average battery life of 5 hours, priced at Rs 80,000.

2. MSI GL65: Intel Core i7 processor running at 2.6 GHz, 16GB RAM, HDD+SSD storage, 15.6" IPS display with a resolution of 1920x1080, NVIDIA GTX graphics card, Windows 10 OS, weighs 2.3 kg, special features include RGB keyboard, 2-year warranty, average battery life of 4 hours, priced at Rs 55,000.

3. Dell Inspiron: Intel Core i5 processor clocked at 2.4 GHz, 8GB RAM, SSD storage, 15.6" LCD display with a resolution of 1920x1080, Intel UHD graphics processor, Windows 10 OS, weighs 2.5 kg, special features include backlit keyboard, 1-year warranty, average battery life of 6 hours, priced at Rs 35,000.



In [58]:
debug_conversation_reco.append({"role": "user", "content": "This is my user profile" + response_dict_n})
debug_conversation_reco.append({"role": "assistant", "content": debug_recommendation})

In [59]:
debug_user_input = "Which is ideal for travel?"

In [60]:
debug_conversation_reco.append({"role": "user", "content": debug_user_input})


In [61]:
debug_response_asst_reco = get_chat_model_completions(debug_conversation_reco)

# Check if debug_response_asst_reco is not None and has 'content' key before printing
if debug_response_asst_reco is not None and 'content' in debug_response_asst_reco:
    print('\n' + debug_response_asst_reco['content'] + '\n')
else:
    print("Error: Could not generate a valid response.")
    print(f"debug_response_asst_reco: {debug_response_asst_reco}") # Print the raw response to debug

Error: Could not generate a valid response.
debug_response_asst_reco: The MSI GL65 is the most portable option among the three laptops. It weighs just 2.3 kg, making it easier to carry around for travel. Despite its lightweight design, it doesn't compromise on performance, offering a powerful Intel Core i7 processor, 16GB of RAM, and a combination of HDD and SSD storage for fast data access. It also features a 15.6" IPS display and an NVIDIA GTX graphics card for an excellent gaming and viewing experience. The laptop also has an average battery life of 4 hours.


# `Areas of improvement`



In [94]:
def get_chat_model_completions_1(messages):
    response = openai.ChatCompletion.create(
        model = "gpt-4-0613",
        messages = messages,
        temperature = 0,
        max_tokens = 500,
        functions = function_descriptions,
        function_call = "auto"
        )

    return response.choices[0]

In [96]:
gamer_conversation = [{"role":"assistant","content":"Hi there! I'm an intelligent laptop gadget expert and I'm here to help you find the best laptop for your needs. Please tell me a little bit about yourself and your requirements so that I can assist you better."},
{"role":"user","content":"Yes, I prefer gaming with my graphics at high setting"},
{"role":"assistant","content":"Thank you for letting me know. Based on your preference for high graphics settings, I would recommend a laptop with a high GPU intensity. This will ensure that you can enjoy your games with excellent visual quality. Now, I have a question to better understand your needs. Are you primarily looking for a laptop that you can easily carry around and game on the go, or do you mainly use it in a stationary location?"},
{"role":"user","content":"I prefer using my laptop at a stationary location"},
{"role":"assistant","content":"Thank you for providing that information. Since you primarily use your laptop in a stationary location, portability may not be a top priority for you. Now, I have one more question to help me understand your requirements better. Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."},
{"role":"user","content":"100000"}
]

gamer_conversation_initialize = initialize_conversation()
print("1.",gamer_conversation_initialize)
gamer_conversation_initialize.extend(gamer_conversation)
print("2.",gamer_conversation_initialize)
gamer_conversation_assistant_output = get_chat_model_completions_1(gamer_conversation_initialize)
print("3.",gamer_conversation_assistant_output)

gamer_response_dict = dictionary_present(gamer_conversation_assistant_output)
print("4.",gamer_response_dict)

gamer_dict = extract_dictionary_from_string(gamer_response_dict)
print("5.",gamer_dict)

1. [{'role': 'system', 'content': '\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You are trying to understand the user\'s requirement for these laptop\'s features: (\'gpu intensity\',\'display quality\',\'portability\',\'multitasking\',\'processing speed\',\'budget\')\n    You need to ask relevant questions and understand the user need for each feature by analysing the user\'s responses.\n    After understanding their requirements, you\'ll use a function call to suggest the top 3 laptops with their respective user match score.\n    Recommend these laptops and answer any user\'s query about them.\n\n    #### Here are certain guidelines that you need to follow:\n    Don\'t ask questions about more than 2 features at a time.\n    If the user\'s budget is less than says less than 25000 INR, please mention that there are no laptops in that range.\n    Recommend the top3 laptops in the following format:\n    Start with a brief sum

In [97]:
gamer_tagged_dict = {'gpu intensity': 'high', 'display quality': 'high', 'portability': 'low', 'multitasking': 'high', 'processing speed': 'high', 'budget': '100000'}
print(gamer_tagged_dict)

{'gpu intensity': 'high', 'display quality': 'high', 'portability': 'low', 'multitasking': 'high', 'processing speed': 'high', 'budget': '100000'}


In [98]:
def evaluate_model_response(tagged_dict, model_dict):
  score = 0
  mappings = {
      'low': 0,
      'medium': 1,
      'high': 2
  }

  for key in tagged_dict.keys():
    if key == 'budget':
      continue

    # Convert key to lowercase before accessing it in model_dict
    tagged_value = tagged_dict[key]
    model_value = model_dict.get(key.lower(), None)  # Use get() to avoid KeyError

    if model_value is None:  # Check if key exists
      print(f"Warning: Key '{key}' not found in model_dict.")
      continue

    tagged_mapping = mappings.get(tagged_value, -1)
    model_mapping = mappings.get(model_value, -1)

    if model_mapping >= tagged_mapping:
        score += 1

  return score

In [99]:
gamer_score = evaluate_model_response(gamer_tagged_dict,gamer_dict)
print(gamer_score)

1


Many more features can be added to this ShopAssist AI project to enhance the functionality.

1. The output format of each layer is inconsistent. You can use the function API capability of GPT to instruct the output format as per the input request.

2. The rule framework provided to classify each laptop’s specification is not exhaustive. You can expand the rules to give a comprehensive context to the LLM.

3. There are misclassifications in the laptop’s specifications, even after specifying clear rules for LLM. You can fine-tune an open-source LLM to make its understanding & performance better.

4. Once the products are extracted, the dialogue flow doesn’t allow recalling of product extraction if there is any intent change. You can add another layer to observe any request for a change in the user intent and then use this flag to recall the product extraction based on the updated intent.

5. As an alternative & simple solution, you can use vector embeddings of each product and compare it with the user intent to find the most relevant products.

6. You can template this workflow/solution to build a chatbot for any product domain.
Note: You must add the relevant domain expertise/rules to give the LLM context understanding.




